In [ ]:
#!wget https://raw.githubusercontent.com/tripathysagar/ODIA_HUMAN_NUMBER/main/odia_digits.csv

--2022-09-06 06:01:18--  https://raw.githubusercontent.com/tripathysagar/ODIA_HUMAN_NUMBER/main/odia_digits.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1025379 (1001K) [text/plain]
Saving to: ‘odia_digits.csv’

odia_digits.csv     100%[===================>]   1001K  --.-KB/s    in 0.05s   

2022-09-06 06:01:18 (20.6 MB/s) - ‘odia_digits.csv’ saved [1025379/1025379]



In [2]:
from fastai.text.all import *
from fastai.imports import *

In [3]:
Path().ls()

(#3) [Path('00.ipynb'),Path('odia_digits.csv'),Path('.ipynb_checkpoints')]

In [4]:
df = pd.read_csv(Path('odia_digits.csv'), low_memory=False)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,10091,10092,10093,10094,10095,10096,10097,10098,10099,10100
0,୦,୧,୨,୩,୪,୫,୬,୭,୮,୯,...,୧୦୦୯୧,୧୦୦୯୨,୧୦୦୯୩,୧୦୦୯୪,୧୦୦୯୫,୧୦୦୯୬,୧୦୦୯୭,୧୦୦୯୮,୧୦୦୯୯,୧୦୧୦୦
1,ଶୁନ୍ୟ,ଏକ,ଦୁଇ,ତିନି,ଚାରି,ପାଞ୍ଚ,ଛଅ,ସାତ,ଆଠ,ନଅ,...,ଦଶ ହଜାର ଏକାନବେ,ଦଶ ହଜାର ବୟାନବେ,ଦଶ ହଜାର ତେୟାନବେ,ଦଶ ହଜାର ଚଉରାନବେ,ଦଶ ହଜାର ପଞ୍ଚାନବେ,ଦଶ ହଜାର ଛୟାନବେ,ଦଶ ହଜାର ସତାନବେ,ଦଶ ହଜାର ଅଠାନବେ,ଦଶ ହଜାର ଅନେଶତ,ଦଶ ହଜାର ଶହେ
2,śunẏa,eka,dui,tini,cāri,pāṅca,chaa,sāta,āṭha,naa,...,daśa hajār ekānabe,daśa hajār baẏānabe,daśa hajār teẏānabe,daśa hajār caurānabe,daśa hajār paṅcānabe,daśa hajār chaẏānabe,daśa hajār satānabe,daśa hajār aṭhānabe,daśa hajār aneśata,daśa hajār śahe


In [5]:
txt = L(df.iloc[1].tolist())

In [6]:
txt

(#10103) ['ଶୁନ୍ୟ','ଏକ','ଦୁଇ','ତିନି','ଚାରି','ପାଞ୍ଚ','ଛଅ','ସାତ','ଆଠ','ନଅ'...]

In [7]:
txt[1000]

'ନଅ ଶହ ସତାନବେ'

In [8]:
text = ' . '.join([l.strip() for l in txt])
text[:100]

'ଶୁନ୍ୟ . ଏକ . ଦୁଇ . ତିନି . ଚାରି . ପାଞ୍ଚ . ଛଅ . ସାତ . ଆଠ . ନଅ . ଦଶ . ଏଗାର . ବାର . ତେର . ଚଉଦ . ପନ୍ଦର . '

Now we need to tokenize the text

In [9]:
tokens = text.split(' ')
tokens[:10]

['ଶୁନ୍ୟ', '.', 'ଏକ', '.', 'ଦୁଇ', '.', 'ତିନି', '.', 'ଚାରି', '.']

We need to fetch all the unique words 

In [10]:
vocab = L(*tokens).unique()
vocab

(#106) ['ଶୁନ୍ୟ','.','ଏକ','ଦୁଇ','ତିନି','ଚାରି','ପାଞ୍ଚ','ଛଅ','ସାତ','ଆଠ'...]

Convert each tokens to numbers

In [11]:
word2idx = {w:i for i,w in enumerate(vocab)}

nums = L(word2idx[w] for w in tokens)
nums

(#54307) [0,1,2,1,3,1,4,1,5,1...]

In [12]:
len(tokens), len(txt)

(54307, 10103)

## Model from scratch

In [13]:
def group_chunks(ds, bs):
    m = len(ds) // bs
    new_ds = L()
    for i in range(m): new_ds += L(ds[i + m*j] for j in range(bs))
    return new_ds

In [14]:
sl = 16
bs = 64
seqs = L((tensor(nums[i:i+sl]), tensor(nums[i+1:i+sl+1]))
         for i in range(0,len(nums)-sl-1,sl))
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs),
                             group_chunks(seqs[cut:], bs),
                             bs=bs, drop_last=True, shuffle=False)

In [15]:
seqs[0]

(tensor([0, 1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1, 7, 1, 8, 1]),
 tensor([1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1, 7, 1, 8, 1, 9]))

In [16]:
x,y = dls.train.one_batch()

In [17]:
x[0],y[0]

(tensor([0, 1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1, 7, 1, 8, 1]),
 tensor([1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1, 7, 1, 8, 1, 9]))

In [18]:
class LMModel1(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        outs = []
        for i in range(sl):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
            outs.append(self.h_o(self.h))
        self.h = self.h.detach()
        return torch.stack(outs, dim=1)
    
    def reset(self): self.h = 0

In [19]:
model =  LMModel1(len(vocab), 64)

In [20]:
model

LMModel4(
  (i_h): Embedding(106, 64)
  (h_h): Linear(in_features=64, out_features=64, bias=True)
  (h_o): Linear(in_features=64, out_features=106, bias=True)
)

In [21]:
ypred = model(x)
ypred.shape

torch.Size([64, 16, 106])

In [22]:
ypred.view(-1, len(vocab)).shape,y.view(-1).shape

(torch.Size([1024, 106]), torch.Size([1024]))

In [23]:

def loss_func(inp, targ):
    return F.cross_entropy(inp.view(-1, len(vocab)), targ.view(-1))

In [24]:
learn = Learner(dls, LMModel1(len(vocab), 64), loss_func=loss_func,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(12, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.323726,3.882199,0.302539,00:01
1,2.902801,2.255141,0.491699,00:01
2,2.160651,2.180123,0.519531,00:01
3,1.785717,2.173019,0.536328,00:01
4,1.558127,2.212782,0.572266,00:01
5,1.361721,2.008071,0.575000,00:01
6,1.214927,2.072815,0.598437,00:01
7,1.112465,2.041016,0.624219,00:01
8,1.029655,2.079587,0.631152,00:01
9,0.976579,2.055110,0.635059,00:01


## using RNN

In [63]:
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

In [64]:
learn = Learner(dls, LMModel2(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.378177,3.808223,0.491211,00:01
1,3.077233,2.087778,0.503711,00:01
2,2.275347,2.136118,0.357812,00:01
3,1.887508,1.935111,0.521191,00:01
4,1.639909,1.972819,0.538086,00:01
5,1.455208,1.934910,0.561621,00:01
6,1.294418,1.947430,0.584766,00:01
7,1.145242,1.884344,0.588672,00:01
8,1.023596,1.877620,0.584180,00:01
9,0.937111,1.877796,0.586230,00:01


## Using LSTM cell


In [27]:
emb = nn.Embedding(len(vocab), 64)

In [28]:
emb(x).shape

torch.Size([64, 16, 64])

In [29]:
len([torch.zeros(10,10) for _ in range(2)])

2

In [71]:
class LMModel3(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.ih = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first = True)
        self.ho = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(n_layers)]
    
    def forward(self, x):
        res, h = self.rnn(self.ih(x), self.h) # returns resedual and hidden state
        #print(res.shape)
        self.h = [h_.detach() for h_ in h]  # delete the derivative part
        
        
        return self.ho(res)
    
    def reset(self):
        for h in self.h: h.zero_()
    

In [72]:
model = LMModel3(len(vocab), 64, 2)
model

LMModel3(
  (ih): Embedding(106, 64)
  (rnn): LSTM(64, 64, num_layers=2, batch_first=True)
  (ho): Linear(in_features=64, out_features=106, bias=True)
)

In [73]:
x.shape, y.shape

(torch.Size([64, 16]), torch.Size([64, 16]))

In [74]:
model(x).shape

torch.Size([64, 16, 106])

In [75]:
learn = Learner(dls, LMModel3(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.081848,3.196913,0.171387,00:02
1,3.193907,2.536864,0.480469,00:02
2,2.339758,2.290623,0.418066,00:03
3,1.969868,2.409519,0.419434,00:02
4,1.767846,2.064037,0.513770,00:02
5,1.597283,2.090909,0.507227,00:02
6,1.436880,2.139480,0.561719,00:03
7,1.330632,2.258411,0.518555,00:02
8,1.247113,2.228344,0.543164,00:02
9,1.158608,2.309157,0.495313,00:02


In [35]:
z = torch.randn(4,5)
z

tensor([[ 0.6994, -0.0688, -0.6969, -1.2239,  0.9241],
        [ 0.4317, -0.3241,  0.7665,  1.2448,  1.1512],
        [ 0.4410, -0.9058,  1.2183, -0.5550,  1.4103],
        [ 0.4232,  1.0480,  0.4265, -0.8930, -0.6336]])

In [36]:
zp = z.new(z.shape)

In [37]:
zp

tensor([[-1.8320e-18,  4.5584e-41,  1.6375e-22,  3.0645e-41,  1.4013e-45],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  2.2813e-22,  3.0645e-41,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  2.8026e-45,  0.0000e+00]])

In [38]:
zpp = z.new(z.shape).bernoulli_(0.8)

In [39]:
zpp

tensor([[1., 1., 0., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 0., 1.]])

In [40]:
z * zpp

tensor([[ 0.6994, -0.0688, -0.0000, -1.2239,  0.9241],
        [ 0.4317, -0.3241,  0.7665,  1.2448,  1.1512],
        [ 0.4410, -0.9058,  1.2183, -0.5550,  1.4103],
        [ 0.4232,  1.0480,  0.4265, -0.0000, -0.6336]])

## LSTM with dropout

In [76]:
class Dropout(Module):
    def __init__(self, p):
        self.p = p
    
    def forward(self,x):
        if not self.training:
            return x
        
        mask = x.new(x.shape).bernoulli_(1-self.p)
        return x * mask.div_(1-self.p)

In [77]:
class LMModel4(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.ih = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first = True)
        self.ho = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(n_layers)]
        
        self.drop = Dropout(p)
    
    def forward(self, x):
        res, h = self.rnn(self.ih(x), self.h) # returns resedual and hidden state
        #print(res.shape)
        res = self.drop(res)
        self.h = [h_.detach() for h_ in h]  # delete the derivative part
        
        
        return self.ho(res)
    
    def reset(self):
        for h in self.h: h.zero_()
    

In [79]:
learn = Learner(dls, LMModel4(len(vocab), 64, 2, .5), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.116068,3.153554,0.177344,00:02
1,3.154489,2.415157,0.399121,00:02
2,2.316478,2.136562,0.415820,00:02
3,1.906218,2.063506,0.489844,00:02
4,1.684795,2.159128,0.502051,00:02
5,1.540506,2.491754,0.521582,00:02
6,1.456420,2.998713,0.509375,00:02
7,1.405630,2.762923,0.509863,00:02
8,1.342089,3.057655,0.519531,00:02
9,1.267954,3.009735,0.521094,00:02


## Weight tied LSTM with dropout

In [80]:
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.ih = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first = True)
        self.ho = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(n_layers)]
        self.ho.weight = self.ih.weight
        self.drop = Dropout(p)
    
    def forward(self, x):
        res, h = self.rnn(self.ih(x), self.h) # returns resedual and hidden state
        #print(res.shape)
        res = self.drop(res)
        self.h = [h_.detach() for h_ in h]  # delete the derivative part
        
        
        return self.ho(res)
    
    def reset(self):
        for h in self.h: h.zero_()
    

In [81]:
learn = Learner(dls, LMModel5(len(vocab), 64, 2, .5), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(12, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.400351,2.550270,0.485645,00:02
1,2.281639,1.957104,0.493066,00:02
2,1.521473,1.889463,0.583691,00:02
3,1.165506,1.879080,0.604785,00:03
4,1.008987,2.022314,0.638867,00:03
5,0.899466,1.743109,0.640039,00:03
6,0.754831,1.474957,0.704980,00:02
7,0.549911,1.233269,0.787109,00:02
8,0.372199,1.076970,0.801562,00:02
9,0.256481,1.027453,0.807617,00:03


## Weight tied LSTM with dropout and Regularization

In [113]:
class LMModel6(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.ih = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first = True)
        
        self.drop = Dropout(p)
        
        self.ho = nn.Linear(n_hidden, vocab_sz)
        self.ho.weight = self.ih.weight
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(n_layers)]
        
        
    
    def forward(self, x):
        raw,h = self.rnn(self.ih(x), self.h)
        out = self.drop(raw)
        self.h = [h_.detach() for h_ in h]
        return self.ho(out),raw,out
        
        
    
    def reset(self):
        for h in self.h: h.zero_()
    

In [114]:
learn = TextLearner(dls, LMModel6(len(vocab), 64, 2, 0.4),
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy)

In [115]:
learn.fit_one_cycle(15, 1e-2, wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,3.444191,2.556527,0.498145,00:02
1,2.008250,1.978638,0.555078,00:02
2,1.349863,1.703701,0.623145,00:02
3,1.050668,1.347721,0.704785,00:02
4,0.697862,0.785333,0.835938,00:02
5,0.393536,0.601545,0.859766,00:02
6,0.219616,0.589667,0.857031,00:02
7,0.128497,0.675003,0.861523,00:02
8,0.081126,0.558522,0.861035,00:02
9,0.056483,0.640782,0.854590,00:02


### Conclusion

The 6th model works well wrt all the other model.